In [30]:
import numpy as np
import pandas as pd 
import tensorflow as tf
import torch
import torch.nn as nn
import torch.nn.functional as F
from nltk.corpus import stopwords 
from collections import Counter
import string
import re
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn.model_selection import GroupKFold, LeaveOneGroupOut
# from torchtext.data import Field, TabularDataset, BucketIterator
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.optim as optim
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import nltk
from nltk.stem import SnowballStemmer
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
import codecs

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\edge\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\edge\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
MAX_NB_WORDS = 50000
# tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english'))
stop_words.update(['.', ',', '"', "'", ':', ';', '(', ')', '[', ']', '{', '}'])

In [3]:
print('loading word embeddings...')

embeddings_index = {}
f = codecs.open('wiki-news-300d-1M.vec', encoding='utf-8')

for line in tqdm(f):
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('found %s word vectors' % len(embeddings_index))

1635it [00:00, 16231.71it/s]

loading word embeddings...


999995it [01:00, 16394.20it/s]

found 999995 word vectors


In [5]:
import random

my_seed = 79
random.seed(my_seed)
np.random.seed(my_seed)
torch.manual_seed(my_seed)
torch.cuda.manual_seed_all(my_seed)

df_raw = pd.read_csv("pheme_english.csv", encoding="utf-8")
# indo_stopwords = ["ada","adalah","adanya","adapun","agak","agaknya","agar","akan","akankah","akhir","akhiri","akhirnya","aku","akulah","amat","amatlah","anda","andalah","antar","antara","antaranya","apa","apaan","apabila","apakah","apalagi","apatah","artinya","asal","asalkan","atas","atau","ataukah","ataupun","awal","awalnya","bagai","bagaikan","bagaimana","bagaimanakah","bagaimanapun","bagi","bagian","bahkan","bahwa","bahwasanya","baik","bakal","bakalan","balik","banyak","bapak","baru","bawah","beberapa","begini","beginian","beginikah","beginilah","begitu","begitukah","begitulah","begitupun","bekerja","belakang","belakangan","belum","belumlah","benar","benarkah","benarlah","berada","berakhir","berakhirlah","berakhirnya","berapa","berapakah","berapalah","berapapun","berarti","berawal","berbagai","berdatangan","beri","berikan","berikut","berikutnya","berjumlah","berkali-kali","berkata","berkehendak","berkeinginan","berkenaan","berlainan","berlalu","berlangsung","berlebihan","bermacam","bermacam-macam","bermaksud","bermula","bersama","bersama-sama","bersiap","bersiap-siap","bertanya","bertanya-tanya","berturut","berturut-turut","bertutur","berujar","berupa","besar","betul","betulkah","biasa","biasanya","bila","bilakah","bisa","bisakah","boleh","bolehkah","bolehlah","buat","bukan","bukankah","bukanlah","bukannya","bulan","bung","cara","caranya","cukup","cukupkah","cukuplah","cuma","dahulu","dalam","dan","dapat","dari","daripada","datang","dekat","demi","demikian","demikianlah","dengan","depan","di","dia","diakhiri","diakhirinya","dialah","diantara","diantaranya","diberi","diberikan","diberikannya","dibuat","dibuatnya","didapat","didatangkan","digunakan","diibaratkan","diibaratkannya","diingat","diingatkan","diinginkan","dijawab","dijelaskan","dijelaskannya","dikarenakan","dikatakan","dikatakannya","dikerjakan","diketahui","diketahuinya","dikira","dilakukan","dilalui","dilihat","dimaksud","dimaksudkan","dimaksudkannya","dimaksudnya","diminta","dimintai","dimisalkan","dimulai","dimulailah","dimulainya","dimungkinkan","dini","dipastikan","diperbuat","diperbuatnya","dipergunakan","diperkirakan","diperlihatkan","diperlukan","diperlukannya","dipersoalkan","dipertanyakan","dipunyai","diri","dirinya","disampaikan","disebut","disebutkan","disebutkannya","disini","disinilah","ditambahkan","ditandaskan","ditanya","ditanyai","ditanyakan","ditegaskan","ditujukan","ditunjuk","ditunjuki","ditunjukkan","ditunjukkannya","ditunjuknya","dituturkan","dituturkannya","diucapkan","diucapkannya","diungkapkan","dong","dua","dulu","empat","enggak","enggaknya","entah","entahlah","guna","gunakan","hal","hampir","hanya","hanyalah","hari","harus","haruslah","harusnya","hendak","hendaklah","hendaknya","hingga","ia","ialah","ibarat","ibaratkan","ibaratnya","ibu","ikut","ingat","ingat-ingat","ingin","inginkah","inginkan","ini","inikah","inilah","itu","itukah","itulah","jadi","jadilah","jadinya","jangan","jangankan","janganlah","jauh","jawab","jawaban","jawabnya","jelas","jelaskan","jelaslah","jelasnya","jika","jikalau","juga","jumlah","jumlahnya","justru","kala","kalau","kalaulah","kalaupun","kalian","kami","kamilah","kamu","kamulah","kan","kapan","kapankah","kapanpun","karena","karenanya","kasus","kata","katakan","katakanlah","katanya","ke","keadaan","kebetulan","kecil","kedua","keduanya","keinginan","kelamaan","kelihatan","kelihatannya","kelima","keluar","kembali","kemudian","kemungkinan","kemungkinannya","kenapa","kepada","kepadanya","kesampaian","keseluruhan","keseluruhannya","keterlaluan","ketika","khususnya","kini","kinilah","kira","kira-kira","kiranya","kita","kitalah","kok","kurang","lagi","lagian","lah","lain","lainnya","lalu","lama","lamanya","lanjut","lanjutnya","lebih","lewat","lima","luar","macam","maka","makanya","makin","malah","malahan","mampu","mampukah","mana","manakala","manalagi","masa","masalah","masalahnya","masih","masihkah","masing","masing-masing","mau","maupun","melainkan","melakukan","melalui","melihat","melihatnya","memang","memastikan","memberi","memberikan","membuat","memerlukan","memihak","meminta","memintakan","memisalkan","memperbuat","mempergunakan","memperkirakan","memperlihatkan","mempersiapkan","mempersoalkan","mempertanyakan","mempunyai","memulai","memungkinkan","menaiki","menambahkan","menandaskan","menanti","menanti-nanti","menantikan","menanya","menanyai","menanyakan","mendapat","mendapatkan","mendatang","mendatangi","mendatangkan","menegaskan","mengakhiri","mengapa","mengatakan","mengatakannya","mengenai","mengerjakan","mengetahui","menggunakan","menghendaki","mengibaratkan","mengibaratkannya","mengingat","mengingatkan","menginginkan","mengira","mengucapkan","mengucapkannya","mengungkapkan","menjadi","menjawab","menjelaskan","menuju","menunjuk","menunjuki","menunjukkan","menunjuknya","menurut","menuturkan","menyampaikan","menyangkut","menyatakan","menyebutkan","menyeluruh","menyiapkan","merasa","mereka","merekalah","merupakan","meski","meskipun","meyakini","meyakinkan","minta","mirip","misal","misalkan","misalnya","mula","mulai","mulailah","mulanya","mungkin","mungkinkah","nah","naik","namun","nanti","nantinya","nyaris","nyatanya","oleh","olehnya","pada","padahal","padanya","pak","paling","panjang","pantas","para","pasti","pastilah","penting","pentingnya","per","percuma","perlu","perlukah","perlunya","pernah","persoalan","pertama","pertama-tama","pertanyaan","pertanyakan","pihak","pihaknya","pukul","pula","pun","punya","rasa","rasanya","rata","rupanya","saat","saatnya","saja","sajalah","saling","sama","sama-sama","sambil","sampai","sampai-sampai","sampaikan","sana","sangat","sangatlah","satu","saya","sayalah","se","sebab","sebabnya","sebagai","sebagaimana","sebagainya","sebagian","sebaik","sebaik-baiknya","sebaiknya","sebaliknya","sebanyak","sebegini","sebegitu","sebelum","sebelumnya","sebenarnya","seberapa","sebesar","sebetulnya","sebisanya","sebuah","sebut","sebutlah","sebutnya","secara","secukupnya","sedang","sedangkan","sedemikian","sedikit","sedikitnya","seenaknya","segala","segalanya","segera","seharusnya","sehingga","seingat","sejak","sejauh","sejenak","sejumlah","sekadar","sekadarnya","sekali","sekali-kali","sekalian","sekaligus","sekalipun","sekarang","sekecil","seketika","sekiranya","sekitar","sekitarnya","sekurang-kurangnya","sekurangnya","sela","selagi","selain","selaku","selalu","selama","selama-lamanya","selamanya","selanjutnya","seluruh","seluruhnya","semacam","semakin","semampu","semampunya","semasa","semasih","semata","semata-mata","semaunya","sementara","semisal","semisalnya","sempat","semua","semuanya","semula","sendiri","sendirian","sendirinya","seolah","seolah-olah","seorang","sepanjang","sepantasnya","sepantasnyalah","seperlunya","seperti","sepertinya","sepihak","sering","seringnya","serta","serupa","sesaat","sesama","sesampai","sesegera","sesekali","seseorang","sesuatu","sesuatunya","sesudah","sesudahnya","setelah","setempat","setengah","seterusnya","setiap","setiba","setibanya","setidak-tidaknya","setidaknya","setinggi","seusai","sewaktu","siap","siapa","siapakah","siapapun","sini","sinilah","soal","soalnya","suatu","sudah","sudahkah","sudahlah","supaya","tadi","tadinya","tahu","tahun","tak","tambah","tambahnya","tampak","tampaknya","tandas","tandasnya","tanpa","tanya","tanyakan","tanyanya","tapi","tegas","tegasnya","telah","tempat","tengah","tentang","tentu","tentulah","tentunya","tepat","terakhir","terasa","terbanyak","terdahulu","terdapat","terdiri","terhadap","terhadapnya","teringat","teringat-ingat","terjadi","terjadilah","terjadinya","terkira","terlalu","terlebih","terlihat","termasuk","ternyata","tersampaikan","tersebut","tersebutlah","tertentu","tertuju","terus","terutama","tetap","tetapi","tiap","tiba","tiba-tiba","tidak","tidakkah","tidaklah","tiga","tinggi","toh","tunjuk","turut","tutur","tuturnya","ucap","ucapnya","ujar","ujarnya","umum","umumnya","ungkap","ungkapnya","untuk","usah","usai","waduh","wah","wahai","waktu","waktunya","walau","walaupun","wong","yaitu","yakin","yakni","yang"]

In [6]:
df = pd.DataFrame({"label": df_raw['label'], "content": df_raw["full_content"], "event": df_raw['event']})

In [7]:
is_cuda = torch.cuda.is_available()
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU is available


In [8]:
# X = df['full_content']
# y = df['label']
# groups = df['event']

In [9]:
# x_train,x_test,y_train,y_test = train_test_split(X,y,stratify=y)
# print(f'shape of train data is {x_train.shape}')
# print(f'shape of test data is {x_test.shape}')

In [10]:
# tokenizer = torchtext.data.utils.get_tokenizer('basic_english')

# def tokenize_data(example, tokenizer, max_length):
#     tokens = tokenizer(example['text'])[:max_length]
#     length = len(tokens)
#     return {'tokens': tokens, 'length': length}

In [11]:
stemmer = SnowballStemmer('english', ignore_stopwords=True)

def tokenize(text):
    tknzr = TweetTokenizer()
    tokens = [word for word in tknzr.tokenize(text)]
    # stems = [stemmer.stem(item) for item in tokens]
    return tokens

In [24]:
logo = LeaveOneGroupOut()
X = df['content']
y = df['label']
groups = df['event']

train_index, test_index = next(logo.split(X, y, groups))
print("TRAIN:", train_index, "TEST:", test_index)
X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index].values, y[test_index].values



TRAIN: [ 458  459  460 ... 6422 6423 6424] TEST: [   0    1    2 ... 4020 4021 4022]


In [25]:
print(X_train)

458     Breaking news: Ghana international and AC Mila...
459     Unconfirmed reports claim that Michael Essien ...
460     BREAKING: Unconfirmed reports claim AC Milan m...
461     Milan have stated that the reports about Essie...
462     AC Milan midfielder Michael Essien has been di...
                              ...                        
6420    Sydney siege ends as police storm Lindt Cafe a...
6421    Breaking News: #SydneySiege is over, according...
6422    Watch gunfire erupt and hostages flee chocolat...
6423    Authorities have confirmed that #sydneysiege i...
6424    WATCH: The dramatic moment tactical teams stor...
Name: content, Length: 4346, dtype: object


In [30]:
# df['doc_len'] = df['content'].apply(lambda words: len(words.split(" ")))
# max_seq_len = 1000
# print(max_seq_len)

1000


In [27]:
max_seq_len = 1000

from sklearn.utils import shuffle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

raw_docs_train = X_train.tolist()
raw_docs_test = X_test.tolist()
num_classes = 4

processed_docs_train = []

for doc in tqdm(raw_docs_train):
    tokens = tokenize(doc)
    filtered = [word for word in tokens if word not in stop_words]
    processed_docs_train.append(" ".join(filtered))
    
processed_docs_test = []

for doc in tqdm(raw_docs_test):
    tokens = tokenize(doc)
    filtered = [word for word in tokens if word not in stop_words]
    processed_docs_test.append(" ".join(filtered))
    
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True, char_level=False)
tokenizer.fit_on_texts(processed_docs_train + processed_docs_test)  #leaky
word_seq_train = tokenizer.texts_to_sequences(processed_docs_train)
word_seq_test = tokenizer.texts_to_sequences(processed_docs_test)
word_index = tokenizer.word_index
print("dictionary size: ", len(word_index))

#pad sequences
word_seq_train = sequence.pad_sequences(word_seq_train, maxlen=max_seq_len)
word_seq_test = sequence.pad_sequences(word_seq_test, maxlen=max_seq_len)


100%|████████████████████████████████████████████████████████████████████████████| 2079/2079 [00:01<00:00, 1498.52it/s]


dictionary size:  90971


In [31]:
y_train = tf.keras.utils.to_categorical(y_train, num_classes = 4)
y_test = tf.keras.utils.to_categorical(y_test, num_classes = 4)

print(y_train)

[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]]


In [40]:
print(X_train)

458     Breaking news: Ghana international and AC Mila...
459     Unconfirmed reports claim that Michael Essien ...
460     BREAKING: Unconfirmed reports claim AC Milan m...
461     Milan have stated that the reports about Essie...
462     AC Milan midfielder Michael Essien has been di...
                              ...                        
6420    Sydney siege ends as police storm Lindt Cafe a...
6421    Breaking News: #SydneySiege is over, according...
6422    Watch gunfire erupt and hostages flee chocolat...
6423    Authorities have confirmed that #sydneysiege i...
6424    WATCH: The dramatic moment tactical teams stor...
Name: content, Length: 4346, dtype: object


In [32]:
#training params
batch_size = 32
num_epochs = 40

#model parameters
num_filters = 16 
embed_dim = 300 
weight_decay = 1e-4

In [35]:
print('preparing embedding matrix...')

words_not_found = []
nb_words = min(MAX_NB_WORDS, len(word_index)+1)
print(nb_words)
embedding_matrix = np.zeros((nb_words, embed_dim))

for word, i in word_index.items():
    if i >= nb_words:
        continue
    embedding_vector = embeddings_index.get(word)
#     print(word)
#     print(embedding_vector)
    if (embedding_vector is not None) and len(embedding_vector) > 0:
        # words not found in embedding index will be all-zeros.
        #print("yes")
        embedding_matrix[i] = embedding_vector
    else:
        words_not_found.append(word)
print(len(words_not_found))

preparing embedding matrix...
50000
26633


In [36]:
from keras.layers import BatchNormalization
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
import tensorflow as tf

model = tf.keras.Sequential()

model.add(Embedding(nb_words,embed_dim,input_length=max_seq_len, weights=[embedding_matrix],trainable=True))
# model.add(Embedding(10000,300,input_length=1000,trainable=True))
model.add(Bidirectional(LSTM(32)))
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(4,activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 1000, 300)         15000000  
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               85248     
 nal)                                                            
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense_3 (Dense)             (None, 4)                 132       
                                                                 
Total params: 15,087,460
Trainable params: 15,087,460
Non-trainable params: 0
__________________________________________

In [41]:
from tensorflow.keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
es_callback = EarlyStopping(monitor='val_loss', patience=3)

train_dataset = tf.data.Dataset.from_tensor_slices((word_seq_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

print(train_dataset)

# Prepare the validation dataset
test_dataset = tf.data.Dataset.from_tensor_slices((word_seq_test, y_test))
test_dataset = test_dataset.batch(64)

with tf.device('/gpu:0'):
    history = model.fit(train_dataset,
          epochs=num_epochs, validation_data=test_dataset, callbacks=[es_callback], shuffle=False)

<BatchDataset shapes: ((None, 1000), (None, 4)), types: (tf.int32, tf.float32)>
Epoch 1/40
68/68 [==============================] - 13s 116ms/step - loss: 1.1109 - accuracy: 0.5446 - val_loss: 0.9711 - val_accuracy: 0.7797
Epoch 2/40
68/68 [==============================] - 8s 111ms/step - loss: 1.4937 - accuracy: 0.5219 - val_loss: 0.8139 - val_accuracy: 0.7797
Epoch 3/40
68/68 [==============================] - 8s 111ms/step - loss: 1.2249 - accuracy: 0.5306 - val_loss: 0.7649 - val_accuracy: 0.7797
Epoch 4/40
68/68 [==============================] - 8s 111ms/step - loss: 1.0039 - accuracy: 0.6420 - val_loss: 0.7669 - val_accuracy: 0.7802
Epoch 5/40
68/68 [==============================] - 8s 111ms/step - loss: 0.7265 - accuracy: 0.7678 - val_loss: 0.8320 - val_accuracy: 0.7744
Epoch 6/40
68/68 [==============================] - 8s 114ms/step - loss: 0.5358 - accuracy: 0.8318 - val_loss: 0.9194 - val_accuracy: 0.7638
